# Tarea 1: Pick & Place con Gripper Flotante

Secuencia completa de pick and place de una banana
sobre la mesa roja usando el gripper Jaco flotante.

Se utiliza la pose de agarre optima obtenida tras evaluar las 10 poses
generadas con GraspIt.

**Requisitos:**
1. Gazebo corriendo: `roslaunch manipulacion_pkg gripper_gazebo.launch tipo_gripper:=jaco objeto:=banana`
2. Fichero YAML con las poses: `grasp_poses/grasp_poses_jaco_banana.yaml`
3. Estructura de archivos:
    - El notebook y `manipulacion_lib.py` en la carpeta `scripts/`.
    - El archivo YAML de poses en `scripts/grasp_poses/grasp_poses_jaco_banana.yaml`.
    - Los modelos URDF y de objetos en sus respectivas carpetas del paquete (`urdf/`, `objects_models/`).

## Parametros de configuracion

Coordenadas de la mesa de destino y alturas de seguridad para el transporte.

In [1]:
# Coordenadas de deposicion sobre la mesa
MESA_X = 0.75
MESA_Y = 0.0

# Geometria de la mesa y margenes
SUPERFICIE_MESA = 0.4    # altura Z de la superficie
DIST_SEGURIDAD  = 0.15   # margen sobre la mesa durante transporte XY
DIST_SOLTAR     = 0.05   # altura sobre la mesa a la que se suelta el objeto

# Parametros de movimiento
OFFSET_PREAGARRE = 0.20  # distancia sobre el objeto para la aproximacion

## Inicializacion

Se inicializa el nodo ROS, se conecta con la simulacion y se abre el
gripper en una posicion segura alejada de los objetos.

In [2]:
import rospy
import rospkg
import yaml
import PyKDL
import os
import manipulacion_lib
from manipulacion_lib import DetectorColisionesGripperFlotante, Obstaculo

# Inicializar nodo (solo si no existe ya)
if not rospy.get_node_uri():
    rospy.init_node('manipulacion_tarea1', anonymous=True)

sim = manipulacion_lib.SimulacionGripperFlotante(nombre_gripper_gazebo="gripper")
sim.configurar_gripper()
rospy.sleep(2.0)
sim.abrir_gripper()
rospy.sleep(3.0)
print("Gripper configurado")

# Posicion inicial del gripper: en el aire, alejado de todo
pose_casa = PyKDL.Frame(PyKDL.Rotation(), PyKDL.Vector(0, 0, 0.5))
sim.fijar_pose_gripper(pose_casa)
rospy.sleep(2.0)
print("Gripper en posicion inicial")

Gripper posicion articulaciones:  [0.1, 0.1, 0.1]
Gripper configurado
Gripper en posicion inicial


## Cargar la pose de agarre

Se carga el fichero YAML con las poses generadas por GraspIt y se
selecciona la pose numero 9 (ordenando por epsilon_quality descendente),
que es la que mejor resultado dio en la fase de entrenamiento.

In [3]:
rospack = rospkg.RosPack()
pkg_path = rospack.get_path('manipulacion_pkg')
yaml_path = os.path.join(pkg_path, 'scripts/grasp_poses/grasp_poses_jaco_banana.yaml')

with open(yaml_path, 'r') as f:
    grasps = yaml.safe_load(f)['grasps']

# Ordenar por epsilon_quality (de mayor a menor) y elegir la pose #9
grasps = sorted(grasps, key=lambda g: g['epsilon_quality'], reverse=True)
INDICE_MEJOR = 8  # indice 8 en lista 0-indexed = pose #9
grasp = grasps[INDICE_MEJOR]

print(f"Pose seleccionada: #{INDICE_MEJOR+1}, epsilon_quality={grasp['epsilon_quality']:.5f}")
print(f"Pose (x,y,z, qx,qy,qz,qw): {[round(v,4) for v in grasp['pose']]}")

Pose seleccionada: #9, epsilon_quality=0.00042
Pose (x,y,z, qx,qy,qz,qw): [-0.0282, 0.0168, 0.1343, -0.6807, -0.5963, -0.3757, 0.2]


## Verificacion de colisiones

Antes de ejecutar el agarre, se comprueba que ni la pose de agarre ni
la pose de aproximacion colisionen con el suelo.

In [4]:
# Detector de colisiones con el suelo
obstaculo_suelo = Obstaculo(
    nombre="suelo", tipo="cubo",
    dimensiones=[4.0, 4.0, 0.01],
    pose=[0, 0, -0.005, 0, 0, 0, 1]
)
detector = DetectorColisionesGripperFlotante('jaco', [obstaculo_suelo])

# Calcular poses en coordenadas mundo
p = grasp['pose']
pose_gripper_obj = PyKDL.Frame(
    PyKDL.Rotation.Quaternion(p[3], p[4], p[5], p[6]),
    PyKDL.Vector(p[0], p[1], p[2])
)
pose_banana = sim.obtener_pose_objeto('banana')
pose_agarre = pose_banana * pose_gripper_obj

vec_sobre = PyKDL.Vector(
    pose_banana.p[0], pose_banana.p[1],
    pose_banana.p[2] + OFFSET_PREAGARRE
)
pose_sobre = PyKDL.Frame(pose_agarre.M, vec_sobre)

# Comprobar colision
q_ag = pose_agarre.M.GetQuaternion()
q_so = pose_sobre.M.GetQuaternion()
col_agarre = detector.hay_colision([pose_agarre.p[0], pose_agarre.p[1], pose_agarre.p[2],
                                    q_ag[0], q_ag[1], q_ag[2], q_ag[3]])
col_sobre  = detector.hay_colision([pose_sobre.p[0], pose_sobre.p[1], pose_sobre.p[2],
                                    q_so[0], q_so[1], q_so[2], q_so[3]])

if col_agarre or col_sobre:
    print("ATENCION: la pose seleccionada colisiona con el entorno")
else:
    print("Sin colisiones, se puede proceder al agarre")

/home/manipulacion/manipulacion_ws/src/manipulacion_pkg/urdf/jaco.urdf
Sin colisiones, se puede proceder al agarre


/home/manipulacion/manipulacion_ws/src/manipulacion_pkg/scripts/manipulacion_lib.py:1015: UserWarning: This function has been marked as deprecated and will be removed in a future release.
  return pin.GeometryObject(


## Fase PICK: agarrar el objeto

Secuencia de agarre:
1. Teletransporte a la posicion de aproximacion (sobre el objeto)
2. Descenso interpolado hasta la pose de agarre
3. Cierre del gripper
4. Ascenso solo en eje Z para verificar que se sujeta el objeto

In [5]:
tipo = sim.get_tipo_gripper()

# 1. Ir a posicion de aproximacion (sobre el objeto)
print("1. Posicionando sobre el objeto")
sim.fijar_pose_gripper(pose_sobre)
rospy.sleep(1.5)

# 2. Descenso interpolado hacia la pose de agarre
print("2. Descendiendo hacia el objeto")
for k in range(50):
    alpha = (k + 1) / 50
    v = pose_sobre.p * (1 - alpha) + pose_agarre.p * alpha
    sim.fijar_pose_gripper(PyKDL.Frame(pose_agarre.M, v))
    rospy.sleep(0.05)

# 3. Cerrar gripper
print("3. Cerrando gripper")
if tipo == 'jaco':
    sim.set_posicion_articulaciones([-0.6, -0.6, -0.6])
rospy.sleep(3.0)

# 4. Ascenso solo en Z (sin cambiar X ni Y)
print("4. Levantando objeto")
ag_x = pose_agarre.p[0]
ag_y = pose_agarre.p[1]
ag_z = pose_agarre.p[2]
z_arriba = pose_sobre.p[2]

for k in range(50):
    alpha = (k + 1) / 50
    z_i = ag_z * (1 - alpha) + z_arriba * alpha
    sim.fijar_pose_gripper(PyKDL.Frame(pose_agarre.M,
                                       PyKDL.Vector(ag_x, ag_y, z_i)))
    rospy.sleep(0.05)
rospy.sleep(0.5)

# Verificar que el objeto se ha levantado
alt_banana = sim.obtener_pose_objeto('banana').p.z()
if alt_banana > 0.05:
    print(f"Pick correcto - banana a z={alt_banana:.3f}m")
else:
    print(f"Pick fallido - banana a z={alt_banana:.3f}m")

1. Posicionando sobre el objeto
2. Descendiendo hacia el objeto
3. Cerrando gripper
Gripper posicion articulaciones:  [-0.6, -0.6, -0.6]
4. Levantando objeto
Pick correcto - banana a z=0.092m


## Fase PLACE: depositar en la mesa

Secuencia de deposicion:
1. Subir a altura de transporte
2. Desplazamiento horizontal hasta encima de la mesa
3. Descenso hasta la altura de soltar
4. Apertura del gripper
5. Retroceso y vuelta a la posicion inicial

Para que el objeto llegue al punto exacto de la mesa, se mide
el offset entre el gripper y el objeto despues del pick y se
compensa la posicion de destino del gripper.

In [6]:
rot_ag = pose_agarre.M

# Calcular alturas de trabajo
altura_soltar   = SUPERFICIE_MESA + DIST_SOLTAR

# Medir offset del objeto respecto al gripper
# Cada agarre sujeta el objeto en una posicion distinta, asi que
# hay que compensar para que el OBJETO (no el gripper) llegue al destino
pos_obj = sim.obtener_pose_objeto('banana').p
off_x = pos_obj[0] - ag_x
off_y = pos_obj[1] - ag_y
off_z = pos_obj[2] - z_arriba

dest_x = MESA_X - off_x      # posicion X del gripper compensada
dest_y = MESA_Y - off_y      # posicion Y del gripper compensada
soltar_z = altura_soltar - off_z   # altura Z del gripper compensada
transp_z = soltar_z + DIST_SEGURIDAD

print(f"Offset objeto-gripper: dx={off_x:.3f}, dy={off_y:.3f}, dz={off_z:.3f}")
print(f"Destino gripper compensado: x={dest_x:.3f}, y={dest_y:.3f}")

# 1. Subir a altura de transporte (solo eje Z)
print("\n1. Subiendo a altura de transporte")
for k in range(80):
    alpha = (k + 1) / 80
    z_i = z_arriba * (1 - alpha) + transp_z * alpha
    sim.fijar_pose_gripper(PyKDL.Frame(rot_ag, PyKDL.Vector(ag_x, ag_y, z_i)))
    rospy.sleep(0.05)
rospy.sleep(0.5)

# 2. Desplazamiento horizontal hacia la mesa (solo ejes X e Y)
print("2. Moviendo hacia la mesa")
for k in range(150):
    alpha = (k + 1) / 150
    x_i = ag_x * (1 - alpha) + dest_x * alpha
    y_i = ag_y * (1 - alpha) + dest_y * alpha
    sim.fijar_pose_gripper(PyKDL.Frame(rot_ag, PyKDL.Vector(x_i, y_i, transp_z)))
    rospy.sleep(0.08)
rospy.sleep(0.3)

# 3. Descenso a la altura de soltar
print("3. Descendiendo sobre la mesa")
for k in range(30):
    alpha = (k + 1) / 30
    z_i = transp_z * (1 - alpha) + soltar_z * alpha
    sim.fijar_pose_gripper(PyKDL.Frame(rot_ag, PyKDL.Vector(dest_x, dest_y, z_i)))
    rospy.sleep(0.05)

# 4. Soltar el objeto
print("4. Soltando objeto")
sim.abrir_gripper()
rospy.sleep(2.0)

# Verificar posicion final
pos_final = sim.obtener_pose_objeto('banana').p
print(f"\nObjeto depositado en: [{pos_final[0]:.3f}, {pos_final[1]:.3f}, {pos_final[2]:.3f}]")

# 5. Retroceder y volver al inicio
print("5. Retrocediendo")
for k in range(30):
    alpha = (k + 1) / 30
    z_i = soltar_z * (1 - alpha) + transp_z * alpha
    sim.fijar_pose_gripper(PyKDL.Frame(rot_ag, PyKDL.Vector(dest_x, dest_y, z_i)))
    rospy.sleep(0.05)

sim.fijar_pose_gripper(pose_casa)
rospy.sleep(2.0)
print("\nTarea completada")

Offset objeto-gripper: dx=0.000, dy=0.028, dz=-0.134
Destino gripper compensado: x=0.750, y=-0.028

1. Subiendo a altura de transporte
2. Moviendo hacia la mesa
3. Descendiendo sobre la mesa
4. Soltando objeto
Gripper posicion articulaciones:  [0.1, 0.1, 0.1]

Objeto depositado en: [0.751, -0.001, 0.325]
5. Retrocediendo

Tarea completada
